In [1]:
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [2]:
# df_bid_ <- all N2 data with library id, concentration, bid
def make_dnase_mnase_samples():
    fp_inp = 'workflows/Worm Accessibility Data Sets - accessibility.tsv'
    fp_out = 'dnase_mnase/dnase_mnase_samples.tsv'
    df_ = pd.read_csv(fp_inp, sep='\t').query('(Strain == "N2") & (Enzyme == "dnase" | Enzyme == "mnase")').sort_values(['Enzyme', 'Stage'])
    df_ = df_.loc[df_['Bioinformatics ID(s)'] == df_['Bioinformatics ID(s)']] # All sequenced libraries
    df_ = df_.loc[~(df_['Library series ID'] == 'EMB-JA0-N2-dnase-test')] # Early test data, partial digestion course
    df_ = df_.loc[~(df_['Library series ID'] == 'EMB-JA11-N2-dnase-S2.1xtraHS082')] # Pippen/beads tests
    df_ = df_.loc[~(df_['Library series ID'] == 'L2-JAX-N2-dnase-SX')] # Pippen/beads tests
    df_ = df_.loc[~(df_['Library series ID'] == 'EMB-JA12-N2-mnase-S2')] # Djem's time course
    df_.reset_index(drop=True).to_csv(fp_out, sep='\t', index=None)
    return df_

df_ = make_dnase_mnase_samples().rename(
    columns={
        'Library series ID': 'libid',
        'Bioinformatics ID(s)': 'bid',
        'Digestion conditions (homogenized)': 'conc',
    }
)[['libid', 'conc', 'bid']]

df_bid_ = pd.DataFrame(
    [(tup.libid, conc, bid) for tup in df_.itertuples() for conc, bid in zip(tup.conc.split(';'), tup.bid.split(';'))],
    columns=['libid', 'conc', 'bid'],
)
print('%d samples total' % (len(df_bid_),))
df_bid_.head()

334 samples total


,libid,conc,bid
0,EMB-JA10-N2-dnase-S1.1,0U_ml,mpg130214_3133
1,EMB-JA10-N2-dnase-S1.1,2.5U_ml,mpg130214_3134
2,EMB-JA10-N2-dnase-S1.1,5U_ml,mpg130214_3135
3,EMB-JA10-N2-dnase-S1.1,10U_ml,mpg130214_3136
4,EMB-JA10-N2-dnase-S1.1,25U_ml,mpg130214_3137


In [3]:
# df_geo_ <- list of libraries for GEO submission
df_geo_ = pd.DataFrame(config['dnase_mnase819_rep']).transpose().reset_index().rename(columns={'index': 'geoid', 'library_id': 'libid'})
df_geo_

,geoid,libid,protocol,source name,title
0,dnase_wt_emb_rep1,EMB-JA10-N2-dnase-S1.2,1,mixed embryos,wt_emb_DNase-seq_rep1
1,dnase_wt_emb_rep2,EMB-JA11-N2-dnase-S2,1,mixed embryos,wt_emb_DNase-seq_rep2
2,dnase_wt_l1_rep1,L1-JA2-N2-dnase-S1.1,1,L1 larvae,wt_L1_DNase-seq_rep1
3,dnase_wt_l1_rep2,L1-JA6-N2-dnase-S2,2,L1 larvae,wt_L1_DNase-seq_rep2
4,dnase_wt_l2_rep1,L2-JA7-N2-dnase-S1,2,L2 larvae,wt_L2_DNase-seq_rep1
5,dnase_wt_l2_rep2,L2-JA1-N2-dnase-S2,2,L2 larvae,wt_L2_DNase-seq_rep2
6,dnase_wt_l3_rep1,L3-JA15-N2-dnase-S1,1,L3 larvae,wt_L3_DNase-seq_rep1
7,dnase_wt_l3_rep2,L3-JA16-N2-dnase-S2,1,L3 larvae,wt_L3_DNase-seq_rep2
8,dnase_wt_l4_rep1,L4-JA3-N2-dnase-S1,2,L4 larvae,wt_L4_DNase-seq_rep1
9,dnase_wt_l4_rep2,L4-JA9-N2-dnase-S2,2,L4 larvae,wt_L4_DNase-seq_rep2


In [4]:
dnase_concentrations = config['dnase_concentrations']
mnase_concentrations = config['mnase_concentrations'] + ['8U_ml_2.4xAMPure']
q_ = '(conc in @dnase_concentrations) | (conc in @mnase_concentrations)'
#q_ = '(conc in @dnase_concentrations)'
df_merge_ = df_bid_.merge(df_geo_, on='libid', how='left').query('geoid == geoid').query(q_).reset_index(drop=True)

# Set 8U_ml_2.4xAMPure as the final 8U sample
assert len(df_merge_.query('conc == "8U_ml_2.4xAMPure"')) == 1
df_merge_.loc[df_merge_.conc == "8U_ml_2.4xAMPure", 'conc'] = "8U_ml"
assert len(df_merge_.query('conc == "8U_ml_2.4xAMPure"')) == 0

for libid_, df_libid_ in df_merge_.groupby('libid'):
    if 'dnase' in libid_:
        missing_ = set(config['dnase_concentrations']) - set(df_libid_['conc'])
        if len(missing_) == 0: missing_ = 'ok'
        print(libid_, '\t', missing_)
    elif 'mnase' in libid_:
        missing_ = set(config['mnase_concentrations']) - set(df_libid_['conc'])
        if len(missing_) == 0: missing_ = 'ok'
        print(libid_, '\t', missing_)

EMB-JA10-N2-dnase-S1.2 	 ok
EMB-JA11-N2-dnase-S2 	 ok
EMB-JA12-N2-mnase-S1 	 {'0.05U_ml', '0.025U_ml'}
EMB-JA13-N2-mnase-S2.2 	 ok
L1-JA2-N2-dnase-S1.1 	 ok
L1-JA6-N2-dnase-S2 	 ok
L2-JA1-N2-dnase-S2 	 ok
L2-JA7-N2-dnase-S1 	 ok
L3-JA15-N2-dnase-S1 	 {'5U_ml'}
L3-JA16-N2-dnase-S2 	 ok
L4-JA3-N2-dnase-S1 	 ok
L4-JA9-N2-dnase-S2 	 ok
YA-JA4-N2-dnase-S1.1 	 {'5U_ml'}
YA-JA5-N2-dnase-S2 	 ok


In [5]:
# Table: samples
df_geo1 = pd.DataFrame()
df_geo1['sample'] = list(map(lambda index: 'Sample %d' % (index + 1,), df_merge_.index))
df_geo1['title'] = list(map(lambda title, conc: '%s_%s' % (title, conc), df_merge_['title'], df_merge_['conc']))
df_geo1['source name'] = df_merge_['source name']
df_geo1['organism'] = 'C. elegans'
df_geo1['characteristics: strain'] = 'wild-type N2'
df_geo1['molecule'] = 'genomic DNA'
df_geo1['protocol'] = df_merge_['protocol']
df_geo1['raw_file_1'] = list(map(lambda geoid, conc: '%s_%s.read1.fastq.gz' % (geoid, conc), df_merge_['geoid'], df_merge_['conc']))
df_geo1['raw_file_2'] = list(map(lambda geoid, conc: '%s_%s.read2.fastq.gz' % (geoid, conc), df_merge_['geoid'], df_merge_['conc']))
df_geo1['bid'] = df_merge_['bid']
df_geo1.to_csv('dnase_mnase819_geo/dnase_mnase_geo1_samples.tsv', index=False, header=True, sep='\t')

In [16]:
# Table: raw files
def md5sum_r1(bid):
    fp = pf(bid, 'md5sum_r1', '.txt', 'dnase_mnase')
    md5 = !cat {fp} | awk '{{print $$1}}'
    return md5[0]

def md5sum_r2(bid):
    fp = pf(bid, 'md5sum_r2', '.txt', 'dnase_mnase')
    md5 = !cat {fp} | awk '{{print $$1}}'
    return md5[0]

def readlen_r1(bid):
    fp = pf(bid, 'readlen_r1', '.txt', 'dnase_mnase')
    readlen = !cat {fp}
    return readlen[0]

def readlen_r2(bid):
    fp = pf(bid, 'readlen_r2', '.txt', 'dnase_mnase')
    readlen = !cat {fp}
    return readlen[0]

df_geo3_r1 = pd.DataFrame()
df_geo3_r2 = pd.DataFrame()
df_geo3_r1['file'] = df_geo1['raw_file_1'].tolist()
df_geo3_r2['file'] = df_geo1['raw_file_2'].tolist()
df_geo3_r1['file_type'] = 'fastq'
df_geo3_r2['file_type'] = 'fastq'
df_geo3_r1['checksum'] = list(map(md5sum_r1, df_geo1['bid']))
df_geo3_r2['checksum'] = list(map(md5sum_r2, df_geo1['bid']))
df_geo3_r1['readlen'] = list(map(readlen_r1, df_geo1['bid']))
df_geo3_r2['readlen'] = list(map(readlen_r2, df_geo1['bid']))

df_geo3_ = pd.concat([df_geo3_r1, df_geo3_r2]).reset_index(drop=True)
df_geo3_['instrument_model'] = 'HiSeq2000'
df_geo3_['single_or_paired'] = 'paired-end'

df_geo3 = df_geo3_[['file', 'file_type', 'checksum', 'instrument_model', 'readlen', 'single_or_paired']]
df_geo3.to_csv('dnase_mnase819_geo/dnase_mnase_geo3_raw.tsv', index=False, header=True, sep='\t')

In [17]:
# Table: paired-end mean/stddev
def fmean(bid):
    fp = pf(bid, 'tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.fsizes', '.txt', 'dnase_mnase')
    df = pd.read_csv(fp, sep='\t', names=['size', 'count'])#.head()
    l_len = list(itertools.chain(*[itertools.repeat(size, count) for (size, count) in zip(df['size'], df['count'])]))
    mean1 = np.average(df['size'], weights=df['count'])
    #mean2 = np.mean(l_len)
    #print(mean1, mean2)
    return '%d' % (mean1,)

def fstd(bid):
    fp = pf(bid, 'tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.fsizes', '.txt', 'dnase_mnase')
    df = pd.read_csv(fp, sep='\t', names=['size', 'count'])#.head()
    l_len = list(itertools.chain(*[itertools.repeat(size, count) for (size, count) in zip(df['size'], df['count'])]))
    return '%d' % (np.std(l_len),)

df_geo4 = pd.DataFrame()
df_geo4['file1'] = df_geo1['raw_file_1']
df_geo4['file2'] = df_geo1['raw_file_2']
df_geo4['avg'] = list(map(fmean, df_geo1['bid']))
df_geo4['std'] = list(map(fstd, df_geo1['bid']))
df_geo4.to_csv('dnase_mnase819_geo/dnase_mnase_geo4_paired_end.tsv', index=False, header=True, sep='\t')

In [21]:
# Table: processed
def processed_rep1_(cond_conc):
    cond, conc = cond_conc
    (assay, stage) = (cond[:5], cond[6:])
    assay = {'dnase': 'DNase-seq', 'mnase': 'MNase-seq'}[assay]
    stage = {'wt_emb': 'Embryo', 'wt_l1': 'L1', 'wt_l2': 'L2', 'wt_l3': 'L3', 'wt_l4': 'L4', 'wt_ya': 'YA',}[stage]
    df_rep1 = df_merge_.query('(geoid == "%s_rep1") & (conc == "%s")' % (cond, conc))
    description = 'Normalized %s %s at %s (replicate 1)' % (stage, assay, conc)
    fp_out = 'dnase_mnase819_geo/tracks/%s_rep1_%s.bw' % (cond, conc)
    fn_out = '%s_rep1_%s.bw' % (cond, conc)
    if len(df_rep1) != 1:
        print('skip: %s, rep1, %s' % (cond, conc))
        return (fn_out, 'bigWig', None, description)
    else:
        step_ = 'tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300'
        fp_inp1 = pf(df_rep1.iloc[0].bid, step_, '_treat_pileup.bw', 'dnase_mnase')
        if not os.path.isfile(fp_out):
            !scripts/bigWiggleTools.ipy write {fp_out} scale 0.1 bin 10 mean {fp_inp1}
        md5_ = !md5sum {fp_out} | awk '{{print $$1}}'
        return (fn_out, 'bigWig', md5_[0], description)

def processed_rep2_(cond_conc):
    cond, conc = cond_conc
    (assay, stage) = (cond[:5], cond[6:])
    assay = {'dnase': 'DNase-seq', 'mnase': 'MNase-seq'}[assay]
    stage = {'wt_emb': 'Embryo', 'wt_l1': 'L1', 'wt_l2': 'L2', 'wt_l3': 'L3', 'wt_l4': 'L4', 'wt_ya': 'YA',}[stage]
    df_rep2 = df_merge_.query('(geoid == "%s_rep2") & (conc == "%s")' % (cond, conc))
    description = 'Normalized %s %s at %s (replicate 2)' % (stage, assay, conc)
    fp_out = 'dnase_mnase819_geo/tracks/%s_rep2_%s.bw' % (cond, conc)
    fn_out = '%s_rep2_%s.bw' % (cond, conc)
    if len(df_rep2) != 1:
        print('skip: %s, rep2, %s' % (cond, conc))
        return (fn_out, 'bigWig', None, description)
    else:
        step_ = 'tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300'
        fp_inp2 = pf(df_rep2.iloc[0].bid, step_, '_treat_pileup.bw', 'dnase_mnase')
        if not os.path.isfile(fp_out):
            !scripts/bigWiggleTools.ipy write {fp_out} scale 0.1 bin 10 {fp_inp2}
        md5_ = !md5sum {fp_out} | awk '{{print $$1}}'
        return (fn_out, 'bigWig', md5_[0], description)

df_geo2 = pd.DataFrame.from_records(
    pmap(processed_rep1_, [('dnase_wt_emb', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('dnase_wt_emb', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep1_, [('dnase_wt_l1', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('dnase_wt_l1', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep1_, [('dnase_wt_l2', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('dnase_wt_l2', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep1_, [('dnase_wt_l3', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('dnase_wt_l3', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep1_, [('dnase_wt_l4', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('dnase_wt_l4', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep1_, [('dnase_wt_ya', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('dnase_wt_ya', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_rep1_, [('mnase_wt_emb', conc) for conc in config['mnase_concentrations']], n_jobs=10) +
    pmap(processed_rep2_, [('mnase_wt_emb', conc) for conc in config['mnase_concentrations']], n_jobs=10)
, columns=['file_name', 'file_type', 'md5', 'description']).query('md5 == md5')
df_geo2.to_csv('dnase_mnase819_geo/dnase_mnase_geo2_processed.tsv', index=False, header=True, sep='\t')
df_geo2

]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_emb_rep1_200U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_emb_rep1_200U_ml.bwb7oym3ou.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg130828_4173.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_emb_rep1_400U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_emb_rep1_400U_ml.bw2br06w29.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg130828_4174.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
chrI	15072423
chrII	15279345
chrIII	13783700
chrIV	17493793
chrV	20924149
chrX	17718866
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_emb_rep1_10U_ml.bw
fp_tmp_chroms=dnase_

[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:   51.3s finished


]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_emb_rep2_400U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_emb_rep2_400U_ml.bwed23357_.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg130705_3284.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_emb_rep2_200U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_emb_rep2_200U_ml.bwa4pkamt6.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg130705_3283.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_emb_rep2_2.5U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_emb_rep2_2.5U_ml.bwfnh4p3gx.tmp.chroms
fp_chroms_template=

[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:   49.9s finished


]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_l1_rep1_25U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_l1_rep1_25U_ml.bwmyz4elt5.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg140522_5038.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_l1_rep1_2.5U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_l1_rep1_2.5U_ml.bwngy92phv.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg140522_5035.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/dnase_wt_l1_rep1_400U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/dnase_wt_l1_rep1_400U_ml.bwwd_xo40v.tmp.chroms
fp_chroms_template=dnase_mn

...

skip: mnase_wt_emb, rep1, 0.05U_ml
skip: mnase_wt_emb, rep1, 0.025U_ml
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/mnase_wt_emb_rep1_16U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/mnase_wt_emb_rep1_16U_ml.bwlq9m014i.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg130809_4061.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/mnase_wt_emb_rep1_4U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/mnase_wt_emb_rep1_4U_ml.bwu6y6iwl_.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg130809_4058.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/mnase_wt_emb_rep1_0.25U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/m

[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:   50.0s finished


]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/mnase_wt_emb_rep2_0.025U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/mnase_wt_emb_rep2_0.025U_ml.bw1vu9dtf9.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg141028_5513.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
]0;IPython: lab/relmappingwt_cmd=write
fp_out=dnase_mnase819_geo/tracks/mnase_wt_emb_rep2_0.05U_ml.bw
fp_tmp_chroms=dnase_mnase819_geo/tracks/mnase_wt_emb_rep2_0.05U_ml.bw0l0erh8l.tmp.chroms
fp_chroms_template=dnase_mnase/tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300/mpg141028_5514.tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300_treat_pileup.bw
chrI	15072423
chrII	15279345
chrIII	13783700
chrIV	17493793
chrV	20924149
chrX	17718866

fp_tmp_wtools=dnase_mnase819_geo/tracks/mnase_wt_emb_rep2_0.025U_ml.bwyq8v6s40.tmp.wig
wiggletools: wiggletools wri

[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:   43.2s finished


,file_name,file_type,md5,description
0,dnase_wt_emb_rep1_2.5U_ml.bw,bigWig,43816436c5890074b6ce29c0507e8649,Normalized Embryo DNase-seq at 2.5U_ml (replic...
1,dnase_wt_emb_rep1_5U_ml.bw,bigWig,757456b191fd9a9d78fb5cb2ad54fdae,Normalized Embryo DNase-seq at 5U_ml (replicat...
2,dnase_wt_emb_rep1_10U_ml.bw,bigWig,6e74bec737144bc4825fb9bbbdfa6bf3,Normalized Embryo DNase-seq at 10U_ml (replica...
3,dnase_wt_emb_rep1_25U_ml.bw,bigWig,5c257e3f0b69b04e4e00a73e33182a50,Normalized Embryo DNase-seq at 25U_ml (replica...
4,dnase_wt_emb_rep1_50U_ml.bw,bigWig,076fe97e48107b95508ea114afdd50e1,Normalized Embryo DNase-seq at 50U_ml (replica...
5,dnase_wt_emb_rep1_100U_ml.bw,bigWig,a454d93c118b447c5b51020071105efa,Normalized Embryo DNase-seq at 100U_ml (replic...
6,dnase_wt_emb_rep1_200U_ml.bw,bigWig,bcd809d2f3caed1f097b0c8a8811429a,Normalized Embryo DNase-seq at 200U_ml (replic...
7,dnase_wt_emb_rep1_400U_ml.bw,bigWig,38178f305968605a10021103d1049c47,Normalized Embryo DNase-seq at 400U_ml (replic...
8,dnase_wt_emb_rep1_800U_ml.bw,bigWig,9ef4ae078d898f27b5aa84e606a680ea,Normalized Embryo DNase-seq at 800U_ml (replic...
9,dnase_wt_emb_rep2_2.5U_ml.bw,bigWig,58c616498dbe9cc85ac75baa950f1a14,Normalized Embryo DNase-seq at 2.5U_ml (replic...


In [5]:
# Table: (deprecated)
"""
def processed_(cond_conc):
    cond, conc = cond_conc
    (assay, stage) = (cond[:5], cond[6:])
    assay = {'dnase': 'DNase-seq', 'mnase': 'MNase-seq'}[assay]
    stage = {'wt_emb': 'Embryo', 'wt_l1': 'L1', 'wt_l2': 'L2', 'wt_l3': 'L3', 'wt_l4': 'L4', 'wt_ya': 'YA',}[stage]
    df_rep1 = df_merge_.query('(geoid == "%s_rep1") & (conc == "%s")' % (cond, conc))
    df_rep2 = df_merge_.query('(geoid == "%s_rep2") & (conc == "%s")' % (cond, conc))
    description = 'Normalized %s %s at %s averaged across replicates' % (stage, assay, conc)
    fp_out = 'dnase_mnase819_geo/tracks/%s_%s.bw' % (cond, conc)
    fn_out = '%s_%s.bw' % (cond, conc)
    if len(df_rep1) != 1 or len(df_rep2) != 1:
        print('skip: %s, %s' % (cond, conc))
        return (fn_out, 'bigWig', None, description)
    else:
        step_ = 'tg_pe.bwa_pe.rm_unmapped_pe.rm_chrM.rm_blacklist.rm_q10.macs2_pe_lt300'
        fp_inp1 = pf(df_rep1.iloc[0].bid, step_, '_treat_pileup.bw', 'dnase_mnase')
        fp_inp2 = pf(df_rep2.iloc[0].bid, step_, '_treat_pileup.bw', 'dnase_mnase')
        if not os.path.isfile(fp_out):
            !scripts/bigWiggleTools.ipy write {fp_out} scale 0.1 bin 10 mean {fp_inp1} {fp_inp2}
        md5_ = !md5sum {fp_out} | awk '{{print $$1}}'
        return (fn_out, 'bigWig', md5_[0], description)

df_geo2 = pd.DataFrame.from_records(
    pmap(processed_, [('dnase_wt_emb', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_, [('dnase_wt_l1', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_, [('dnase_wt_l2', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_, [('dnase_wt_l3', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_, [('dnase_wt_l4', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_, [('dnase_wt_ya', conc) for conc in config['dnase_concentrations']], n_jobs=10) +
    pmap(processed_, [('mnase_wt_emb', conc) for conc in config['mnase_concentrations']], n_jobs=10)
, columns=['file_name', 'file_type', 'md5', 'description']).query('md5 == md5')
df_geo2.to_csv('dnase_mnase819_geo/dnase_mnase_geo2_processed.tsv', index=False, header=True, sep='\t')
"""

[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    1.8s finished
[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    1.6s finished
[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    2.3s finished


skip: dnase_wt_l3, 5U_ml


[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    1.6s finished
[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    2.0s finished


skip: dnase_wt_ya, 5U_ml


[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    2.8s finished


skip: mnase_wt_emb, 0.025U_ml
skip: mnase_wt_emb, 8U_ml
skip: mnase_wt_emb, 0.05U_ml


[Parallel(n_jobs=10)]: Done   9 out of   9 | elapsed:    2.0s finished
